In [1]:
import pandas as pd
from histr import Shabdansh

In [2]:
names = pd.read_csv('../naam.csv')

In [3]:
names['name'] = names['name'].map(Shabdansh)

In [4]:
# remove names with only one syllable
names = names[names['name'].map(len) > 1]

In [5]:
words = names.sort_values(by=['count'], ascending=False)['name'][:32000].tolist()

/tmp/ipykernel_1848885/4186441724.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  words = names.sort_values(by=['count'], ascending=False)['name'][:32000].tolist()


In [8]:
print(f"Min Syllable Count: {min(len(w) for w in words)}, Max Syllable Count: {max(len(w) for w in words)}")

Min Syllable Count: 2, Max Syllable Count: 10


In [9]:
b = {}
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2 in zip(chs, chs[1:]):
    bigram = (ch1, ch2)
    b[bigram] = b.get(bigram, 0) + 1

In [10]:
sorted(b.items(), key = lambda kv: -kv[1])

[(('न', '<E>'), 3391),
 (('ह', '<E>'), 3223),
 (('र', '<E>'), 2927),
 (('ल', '<E>'), 2430),
 (('रा', 'म'), 2135),
 (('सिं', 'ह'), 1921),
 (('<S>', 'रा'), 1834),
 (('<S>', 'अ'), 1647),
 (('म', '<E>'), 1308),
 (('<S>', 'स'), 1234),
 (('ला', 'ल'), 1180),
 (('सि', 'ह'), 1105),
 (('द', '<E>'), 1104),
 (('<S>', 'म'), 1017),
 (('<S>', 'ज'), 847),
 (('<S>', 'सु'), 832),
 (('<S>', 'र'), 796),
 (('<S>', 'ह'), 752),
 (('पा', 'ल'), 736),
 (('री', '<E>'), 724),
 (('ली', '<E>'), 691),
 (('<S>', 'ब'), 668),
 (('<S>', 'वि'), 646),
 (('<S>', 'क'), 642),
 (('<S>', 'न'), 634),
 (('त', '<E>'), 629),
 (('<S>', 'मु'), 620),
 (('श', '<E>'), 615),
 (('रा', 'ज'), 554),
 (('कु', 'मा'), 538),
 (('ई', '<E>'), 533),
 (('<S>', 'इ'), 517),
 (('ती', '<E>'), 508),
 (('ज', '<E>'), 496),
 (('मा', 'र'), 490),
 (('सा', 'द'), 486),
 (('प्र', 'सा'), 474),
 (('न्द्र', '<E>'), 454),
 (('क', 'र'), 445),
 (('<S>', 'शि'), 435),
 (('दी', 'न'), 435),
 (('<S>', 'प'), 420),
 (('र', 'न'), 420),
 (('रा', 'य'), 419),
 (('थ', '<E>'), 41

In [12]:
import torch

In [13]:
chars_ls = [list(word) for word in words]
chars = []
for ls in chars_ls:
    chars += ls

In [18]:
purna_virama = '।'
chars = sorted(list(set(chars)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi[purna_virama] = 0
itos = {i:s for s,i in stoi.items()}

In [19]:
unique_syallable_count = len(stoi)

In [20]:
N = torch.zeros((unique_syallable_count, unique_syallable_count), dtype=torch.int32)

In [21]:
for w in words:
  chs = [purna_virama] + list(w) + [purna_virama]
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [22]:
p = N[0].float()
p = p / p.sum()
p

tensor([0.0000e+00, 5.1469e-02, 9.3750e-05,  ..., 3.1250e-05, 6.2500e-05,
        3.1250e-05])

In [23]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

'र'

In [24]:
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

tensor([0.6064, 0.3033, 0.0903])

In [63]:
torch.multinomial(p, num_samples=100, replacement=True, generator=g)

tensor([1, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 2, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0,
        0, 1, 1, 1])

In [64]:
p.shape

torch.Size([3])

In [26]:
P = (N+1).float()
P /= P.sum(1, keepdims=True)

In [28]:
P.shape

torch.Size([1486, 1486])

In [29]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

रवार्मेंक्मिक्रेसश्फाक्ष्मिम्ति्यायंम्भन्यस्कम्ब्रीशेाल्तीहुँप्तथ्रव्तेक्शून्हेंल्फिनौंफ्तेर्मीचिंधुल्हुच्चीतृछूआैंम्भिपौफ्फासाेन्यार्फूहेाचोल्तामैन।
ल।
चिंर्भनंेभ्ॉडूधांछेढ़ेत्माढवेंन्तचेद्याीठोन्टीरांर्तीम्पान्टेशेफोदौम्वण्डतौत्यस्मीश्यन्ठील्फअादोंछूघाष्कआेगूछंनीीन्वास्सुल्तगेंधैापॉद्रिबृजंंब्बव्याध्यील्ल्नम्हुढ़ढ़ूम्देय्यूद्रौरि।
कृढिक्सतूकाओंन्छार्जान्थामाीबूीन्जाग्गुकुँष्मलाशवकरतत्पाप्तील्टील्टूर्नाष्यचीढ़ुहूटीछौष्मितींश्नाम्ब्रीब्रेन्ताेच्छैभोछूपेश्याहाुव्लूनीीस्फाषीरोंम्बोध्नडुफुर्जीम्मीम्बोभंठुष्ठर्इंर्मेशद।
मवटीीत्निव्रश्त्याड़तीीन्धारोंल्फिस्थास्मीबूीर्रावैघूवॅव्नेम्भूमुॅट्ठग्निखैस्कूछोदुग्रेबंल्लूण्डूध्रूकैधेानाीत्तम्ति्याप्रान्हूँभीम्तेन्हूंन्शुन्थून्दिक्ताफेंयिच्चीम्पिस्मीआैण्डिर्फाहुँब्लीश्यकांहून्ताडँूद्धद्वदाब्वक्कान्हूढोस्वाहूल्फेल्फन्डैरोस्मील्हेम्याूल्पूट्टूल्डीक्रुश्याक्केल्हरेाच्छैक्सीपंंश्बल्हून्डोन्ध्ययिस्पमुँल्हूढ़ुम्वोर्जाट्रड्डेमीराीत्तूढोंन्हूॅन्साशैक्मट्ठमोैन्द्रोरॅद्धिचर्वगंडोपाँडूँल्ल्नबोद्धित्रिढोश्वात्तनंेश्फाचंन्कीलाल।


In [69]:
# GOAL: maximize likelihood of the data w.r.t. model parameters (statistical modeling)
# equivalent to maximizing the log likelihood (because log is monotonic)
# equivalent to minimizing the negative log likelihood
# equivalent to minimizing the average negative log likelihood

# log(a*b*c) = log(a) + log(b) + log(c)

In [30]:
log_likelihood = 0.0
n = 0

for w in words:
  chs = [purna_virama] + list(w) + [purna_virama]
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-653013.6875)
nll=tensor(653013.6875)
4.051154613494873


In [42]:
# create the training set of bigrams (x,y)
xs, ys = [], []

for w in words:
  chs = [purna_virama] + list(w) + [purna_virama]
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [45]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=len(itos)).float()
xenc

tcmalloc: large alloc 1916256256 bytes == 0xc81c000 @  0x7f772e931680 0x7f772e952824 0x7f772e952b8a 0x7f75c9096dc5 0x7f75c9074053 0x7f75f701027f 0x7f75f70110a0 0x7f75f70110f4 0x7f75f76b213f 0x7f75f810138b 0x7f75f7dc1c87 0x7f75f80d3faf 0x7f75f7e02b62 0x7f75f76b88a6 0x7f75f828dda5 0x7f75f7af86e2 0x7f75f80d4245 0x7f75f7b55641 0x7f75f7545d07 0x7f75f844bb40 0x7f75f7b326a4 0x7f76211d07b5 0x53a079 0x62519c 0x5aca00 0x5a61d1 0x6d56ef 0x6444a1 0x53933f 0x5a71fb 0x651137


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [46]:
xenc.shape

torch.Size([161192, 1486])

In [36]:
xenc.dtype

torch.float32

In [40]:
xenc.shape

torch.Size([3, 1486])

In [47]:
W = torch.randn((unique_syallable_count, 1))
xenc @ W

tensor([[ 1.1912],
        [ 0.2513],
        [-0.8590],
        ...,
        [-2.5520],
        [ 1.4006],
        [-0.6807]])

In [48]:
logits = xenc @ W # log-counts
counts = logits.exp() # equivalent N
probs = counts / counts.sum(1, keepdims=True)
probs

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])

In [81]:
probs[0]

tensor([1.])

In [82]:
probs[0].shape

torch.Size([1])

In [507]:
probs[0].sum()

tensor(1.)

In [ ]:
# (5, 27) @ (27, 27) -> (5, 27)

In [ ]:
# SUMMARY ------------------------------>>>>

In [528]:
xs

tensor([ 0,  5, 13, 13,  1])

In [529]:
ys

tensor([ 5, 13, 13,  1,  0])

In [49]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((unique_syallable_count, unique_syallable_count), generator=g)

In [50]:
xenc = F.one_hot(xs, num_classes=unique_syallable_count).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# btw: the last 2 lines here are together called a 'softmax'

tcmalloc: large alloc 1916256256 bytes == 0xc81c000 @  0x7f772e931680 0x7f772e952824 0x7f772e952b8a 0x7f75c9096dc5 0x7f75c9074053 0x7f75f701027f 0x7f75f70110a0 0x7f75f70110f4 0x7f75f76b213f 0x7f75f810138b 0x7f75f7dc1c87 0x7f75f80d3faf 0x7f75f7e02b62 0x7f75f76b88a6 0x7f75f828dda5 0x7f75f7af86e2 0x7f75f80d4245 0x7f75f7b55641 0x7f75f7545d07 0x7f75f844bb40 0x7f75f7b326a4 0x7f76211d07b5 0x53a079 0x62519c 0x5aca00 0x5a61d1 0x6d56ef 0x6444a1 0x53933f 0x5a71fb 0x651137


In [51]:
probs.shape

torch.Size([161192, 1486])

In [52]:

nlls = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

--------
bigram example 1: ।रा (indexes 0,1034)
input to the neural net: 0
output probabilities from the neural net: tensor([0.0019, 0.0003, 0.0004,  ..., 0.0004, 0.0001, 0.0017])
label (actual next character): 1034
probability assigned by the net to the the correct character: 0.0005312002031132579
log likelihood: -7.540371417999268
negative log likelihood: 7.540371417999268
--------
bigram example 2: राम (indexes 1034,918)
input to the neural net: 1034
output probabilities from the neural net: tensor([3.7889e-04, 8.3393e-05, 5.1470e-04,  ..., 2.5947e-04, 4.8348e-05,
        4.0333e-03])
label (actual next character): 918
probability assigned by the net to the the correct character: 0.0006477683200500906
log likelihood: -7.341977596282959
negative log likelihood: 7.341977596282959
--------
bigram example 3: म। (indexes 918,0)
input to the neural net: 918
output probabilities from the neural net: tensor([0.0001, 0.0005, 0.0004,  ..., 0.0004, 0.0001, 0.0008])
label (actual next character

In [561]:
# --------- !!! OPTIMIZATION !!! yay --------------

In [53]:
xs

tensor([   0, 1034,  918,  ...,  849,  526, 1376])

In [54]:
ys

tensor([1034,  918,    0,  ...,  526, 1376,    0])

In [55]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((unique_syallable_count, unique_syallable_count), generator=g, requires_grad=True)

In [57]:
# forward pass
xenc = F.one_hot(xs, num_classes=unique_syallable_count).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
loss = -probs[torch.arange(xs.shape[0]), ys].log().mean()

tcmalloc: large alloc 1916256256 bytes == 0x248188000 @  0x7f772e931680 0x7f772e952824 0x7f772e952b8a 0x7f75c9096dc5 0x7f75c9074053 0x7f75f701027f 0x7f75f70110a0 0x7f75f70110f4 0x7f75f76b213f 0x7f75f810138b 0x7f75f7dc1c87 0x7f75f80d3faf 0x7f75f7e02b62 0x7f75f76b88a6 0x7f75f828dda5 0x7f75f7af86e2 0x7f75f80d4245 0x7f75f7b55641 0x7f75f7545d07 0x7f75f844bb40 0x7f75f7b326a4 0x7f76211d07b5 0x53a079 0x62519c 0x5aca00 0x5a61d1 0x6d56ef 0x6444a1 0x53933f 0x5a71fb 0x651137


In [58]:
print(loss.item())

7.73654317855835


In [59]:
# backward pass
W.grad = None # set to zero the gradient
loss.backward()

In [60]:
W.data += -0.1 * W.grad

In [61]:
# --------- !!! OPTIMIZATION !!! yay, but this time actually --------------

In [66]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = [purna_virama] + list(w) + [purna_virama]
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((unique_syallable_count, unique_syallable_count), generator=g, requires_grad=True)

number of examples:  161192


In [75]:
# finally, sample from the 'neural net' model
def generate_names(W: torch.tensor, name_count: int = 5) -> None:
    g = torch.Generator().manual_seed(2147483647)
    for i in range(name_count):
        out = []
        ix = 0
        while True:
            xenc = F.one_hot(torch.tensor([ix]), num_classes=unique_syallable_count).float()
            logits = xenc @ W # predict log-counts
            counts = logits.exp() # counts, equivalent to N
            p = counts / counts.sum(1, keepdims=True) # probabilities for next character
            ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
            out.append(itos[ix])
            if ix == 0:
                break
        print(''.join(out))

In [77]:
# gradient descent
for k in range(10000):
    # forward pass
    xenc = F.one_hot(xs, num_classes=unique_syallable_count).float() # input to the network: one-hot encoding
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()

    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()

    # update
    W.data += -50 * W.grad
    
    if k%100 == 0:
        print(f"At step: {k}, loss: {loss}")
        generate_names(W, 5)


At step: 0, loss: 4.624068737030029
रायन्धेक्कूक्शूष्मष्ठखंम्तिम्मूयेन्नस्नाम्ताशैाल्टीहुंफित्थीविाक्षिन्सील्टुनीीभोंर्बीचन्दर।
त्येंछिंऊंम्दन्हूँब्दुसेन्घर्फुहुँछंाल्फीम्खांन्जेक्टर्धच्चूटीीर्भुधोमंंड़धोंज्जोट्टोत्त्ूड़िल्ल्ून्तेच्चिद्दूडूंन्घरंर्मीम्भिन्छीशौप्ॉद्धीम्पोतींति।
स्मिश्विन्ठल्थूअॅद्धेघूघुषेाआेंगोैजिनेान्सीहंल्ठूगीदेवत्त्ूप्रेटुब्बूशेनैल्पूयांण्डीढूम्मूर्चिद्धायूआंग्रेड़ीख्तत्थ्ूक्टूक्ष्मिनप्तैन्दृमाीब्दीन्कूगेंकुँष्यंल्टूश्व्नाश्कखफ्फून्द्रीन्टपरमेर्धष्कचॉण्ठीहेाट्ठछेषित्तिश्रामुॅबेन्ड़ेछिमंंछपंश्रहाेव्दुद्योस्माष्नारिेम्बुध्नड्डोप्रेम्हराूम्ताबिाड़ाष्णर्इंर्मीवेन्त।
मल्पाझातिर्योश्चंडेत्थेन्दृर्गील्लोस्तास्थाभेार्रव्दाचंव्यवोम्समंुठिग्गूखिस्कूजंेधांगैाबूंल्दूत्थुध्दुकात्मध्वत्युम्भीबिाप्तीबॉम्यापँन्सोन्ठून्द्रीक्तिप्रायिघौम्बस्थिई।
र्वहोैब्ॉशँकृहुंन्तूड्डेद्दिद्ददाीप्पीकेापालसौहॅल्पल्टून्त्ररंस्टवेभंम्हुल्मिड्डोलेाक्शीश्राकुँल्लूरैच्चखाीफुशोैवून्द्रक्षीरीीस्लाभृल्हौतापर।
At step: 100, loss: 4.4132184982299805
राम्म्ार्जीक्खोक्षसौश्ॉक्सूम्ताम्भयंन्नोसौमोैशूल्दीहॅन्सोथुव्र

At step: 4400, loss: 3.4326162338256836
रवाल।
इसविचाप्रिंयाीयोेन्ध्यासृम्वरमुह।
देवा।
बेल।
मल।
At step: 4500, loss: 3.427898406982422
रवाल।
इसविचाप्रीतबबिहायर्जन्हाहाँप्पुदूल।
बेल।
मल।
निशा।
At step: 4600, loss: 3.4233205318450928
रवंश।
इसविचाप्रेमब्रान्नीीस्कूम्म्दशोैल्लीप्रसादर।
बेल।
मल।
निशा।
At step: 4700, loss: 3.418874502182007
रवंश।
इसविचाप्रेमब्रम्हाष्यंम्बूव्लूल्लुहीमजल।
बेल।
मल।
निशा।
At step: 4800, loss: 3.4145545959472656
रवंश।
इसविचाप्रेमब्बोन्तिसोेम्याशेालह।
देवा।
बेर्सन्।
ल।
At step: 4900, loss: 3.41035532951355
रवंश।
इसविचाप्ॉय्यम्ति्यान्मुसॉय्युव्वून्नीसिंह।
शितटोली।
मल।
निशा।
At step: 5000, loss: 3.4062716960906982
रवंश।
इसविचाप्रेमब्बू।
सुमेश्वरीसिंह।
शितटोली।
मल।
At step: 5100, loss: 3.4022977352142334
रवंश।
इसविचाप्रेमब्बुन्सासँम्बेश्त्याल्पूहुॅप्रेमशी।
बेर्वेशन्कर।
निशा।
ता।
At step: 5200, loss: 3.3984298706054688
रवंश।
इसविचाप्रेमब्बिन्हाष्पम्पोशंकसूर।
शितट्टरकरबिजनराजगट्ठुआलई।
सुन्दरह।
विमति।
At step: 5300, loss: 3.394662380218506
रवंश।
इसविचाप्रेमब्बप्ॉस्सिम्भे

KeyboardInterrupt: 